In [1]:
import os
import tqdm
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import sklearn.cluster 
import pandas as pd
from sklearn import preprocessing
from time import time # 用于计算运行时间
import matplotlib.pyplot as plt 
import numpy as np
import cv2,os,tqdm
from matplotlib import offsetbox # 定义图形box的格式
from matplotlib.pyplot import MultipleLocator
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection) 

Image.MAX_IMAGE_PIXELS = None

##  聚类算法  sklearn

#### 单张图片像素数聚类

In [ ]:
def img_count_pixel(img_file,save_file):
    if not os.path.exists(save_file):
        os.makedirs(save_file)
    for name in tqdm.tqdm(os.listdir(img_file)):
        img_path = os.path.join(img_file,name)
        img = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
        img_copy = img
        pixel_list = []
        df1_dict = {}
        for num in range(256):
            pixel_list.append(np.sum(img==num))
        df1_dict[str(name.split('.')[0])] = pixel_list
        data=pd.DataFrame(df1_dict)
        data = data.drop([0])
        data = data[~(data.isin([0]))]
        data = data.dropna(axis=0,how='any')
        data = data.astype(int)
        data_index = np.array(data.index)
        # 由于几个特征的量纲差距很大，聚类前必须进行归一化
        cluster_data = preprocessing.scale(data)
        # data数据进行归一化
        cluster_data = cluster_data.reshape(-1,1)
        min_max_scaler = preprocessing.MinMaxScaler()
        cluster_data = min_max_scaler.fit_transform(cluster_data)
        cluster_result = sklearn.cluster.AgglomerativeClustering(n_clusters=3).fit(cluster_data)
        img_class = cluster_result.labels_
        img_class[np.where(img_class==[0])]=[3]
        img_class.reshape(1,-1)
        for i,j in enumerate(data_index):
            img_copy[np.where(img_copy == [j])] = img_class[i]
        cv2.imwrite(os.path.join(save_file,name),img_copy)

img_file  = r'D:\software\Code\code-file\Preprocess\img_test2\3_single_img\2\Lab\b_channel'
save_file = r'D:\software\Code\code-file\Preprocess\img_test2\4_index_map\2_single\Lab\b_channel'
img_count_pixel(img_file,save_file)   

#### 像素总数进行聚类

In [ ]:
def pixel_sum(input_file,save_file,nnumbe_class=5):
    if not os.path.exists(save_file):
        os.makedirs(save_file)
    #创建Dataframe字典
    df_dict = {}
    for name in tqdm.tqdm(os.listdir(input_file)):
        img_path = os.path.join(input_file,name)
        img = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
        img_list = []
        for i in range(256):  
            img_list.append(np.sum(img==i))
        #保存到字典
        df_dict[str(name.split('.')[0])] = img_list 
    #使用字典创建DataFrame
    df=pd.DataFrame(df_dict)
    #是否统计总值
    df['all'] = df.apply(lambda x:x.sum(),axis =1)
    data = df['all']
    data = data.drop([0])
    data = data[~(data.isin([0]))]
    data = data.dropna(axis=0,how='any')
    data = data.astype(int)
    data_index = np.array(data.index)
    # 由于几个特征的量纲差距很大，聚类前必须进行归一化
    cluster_data = preprocessing.scale(data)
    # data数据进行归一化
    cluster_data = cluster_data.reshape(-1,1)
    min_max_scaler = preprocessing.MinMaxScaler()
    cluster_data = min_max_scaler.fit_transform(cluster_data)
    cluster_result = sklearn.cluster.AgglomerativeClustering(n_clusters=nnumbe_class).fit(cluster_data)
    img_class = cluster_result.labels_
    img_class[np.where(img_class==[0])]=[nnumbe_class]
    img_class.reshape(1,-1)
    for name in tqdm.tqdm(os.listdir(input_file)):
        img_path = os.path.join(input_file,name)
        img = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE)
        for i,j in enumerate(data_index):
            img[np.where(img == [j])] = img_class[i]
        cv2.imwrite(os.path.join(save_file,name),img)


input_file = r'D:\31890\Pictures\image\Root_senescence\channel_image\2\hsv\s_channel'
save_file = r'D:\31890\Pictures\image\Root_senescence\clustering_image\2\hsv\s_channel'
pixel_sum(input_file,save_file)  

#### 单通道位置像素聚类

In [ ]:
#单通道位置聚类的索引图的制作
def Color_Indexes(input_file,save_file):
    if not os.path.exists(save_file):
                os.makedirs(save_file)
    '''     input_file:单通道输入图片
            save_file:保存图片的路径   '''

    for img_name in tqdm.tqdm(os.listdir(input_file)):
            image_path = os.path.join(input_file,img_name)
            img_copy = 0
            #单通道读取图片
            img = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
            img_copy = img
            #提取非0元素进行聚类数据
            tmp = img_copy[img_copy != [0]]
            #数据整理
            tmp = tmp.reshape(-1,1)
            #数据归一化(对非0数据无法进行聚类)
            min_max_scaler = preprocessing.MinMaxScaler()
            cluster_data = min_max_scaler.fit_transform(tmp)
            #聚类
            #MiniBatchKMeans适用于数据较大
            cluster_result = sklearn.cluster.MiniBatchKMeans(n_clusters=3,batch_size=4096,n_init=10).fit(cluster_data)
            #聚类结果
            res = cluster_result.labels_
            #替换聚类结果中的0元素
            res[np.where(res==[0])]=[3]
            img_copy[np.where(img_copy != [0])] = res

            #保存路径
            save_path = os.path.join(save_file,img_name)
            cv2.imwrite(save_path,img_copy)

input_file = r'D:\software\Code\code-file\Preprocess\img_test2\3_single_img\2\Lab\L_channel'     
save_file = r'D:\software\Code\code-file\Preprocess\img_test2\4_index_map\2_position\Lab\L_channel'       
Color_Indexes(input_file,save_file)

#### 其他

In [ ]:
#对像素总和聚类
def cluster_pixel(xlsx):
    df_ = pd.read_excel(xlsx,index_col=0)
    #读取总和像素
    data = df_['all'].astype(int)

    #丢弃掉0行和无像素值的行
    data = data.drop([0])
    data = data[~(data.isin([0]))]
    data = data.dropna(axis=0,how='any')
    data = data.astype(int)
    data = data.rename('0')

    #获取行索引
    index_list= data.index.tolist()
    # 由于几个特征的量纲差距很大，聚类前必须进行归一化
    cluster_data = preprocessing.scale(data)
    # data数据进行归一化
    cluster_data = cluster_data.reshape(-1,1)
    min_max_scaler = preprocessing.MinMaxScaler()
    cluster_data = min_max_scaler.fit_transform(cluster_data)
    #聚类
    # cluster_result = AgglomerativeClustering(n_clusters=8).fit(cluster_data)
    # # cluster_result = OPTICS(min_samples=6).fit(cluster_data)
    # # cluster_result = SpectralClustering(n_clusters=8,
    # #     assign_labels='discretize',
    # #     random_state=0).fit(cluster_data)
    # # cluster_result = KMeans(n_clusters=6, ).fit(cluster_data)
    # # 将原数据和聚类结果输出excel
    # cluster_dict = {'cluster':cluster_result.labels_}
    # df_cluster = pd.DataFrame(cluster_dict,index=index_list)
    # df_result = pd.concat([df_,df_cluster],axis=1)
    # df_result = df_result.fillna(0).astype(int)
    # df_result.to_excel(r'D:\SoftWare\Code\Code-file\Preprocess\4.xlsx')
    
xlsx_path =r'D:\SoftWare\Code\Code-file\Preprocess\3_image_test\4_xlsx_file\2\BGR\B_channel.xlsx'
cluster_pixel(xlsx_path)

## sk超像素

#### skimage

In [ ]:
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float

def Superpixel_segmentation(image_array,n_class):
    segments = slic(img_as_float(image_array), n_segments=n_class, sigma=10)
    result = mark_boundaries(image_array,segments)*255
    return result

def Superpixel_averaging(image_array,n_class):
    segments = slic(img_as_float(image_array), n_segments=n_class, sigma=10)
    seg_class = segments.max()
    color_array = np.zeros((seg_class,3))
    for i in range(seg_class):
        class_array = image_array[segments==i+1]
        color_array[i] = np.mean(class_array,axis=0)
    color_array = color_array.astype(np.int32)
    result = color_array[(segments-1).flatten()]
    result = result.reshape(image_array.shape)
    return result

#单张图像
image_path = r"D:\software\Code\code-file\pytorch-template\test\cmask.png"
image_array = cv2.imread(image_path)
image_array = cv2.cvtColor(image_array,cv2.COLOR_BGR2RGB)
result = Superpixel_segmentation(image_array,900)
result = cv2.cvtColor(result,cv2.COLOR_RGB2BGR)
cv2.imwrite(r'D:\software\Code\code-file\pytorch-template\test\cmask_sub.png',result)

# #系列图像
# image_file = r"D:\31890\Pictures\image\Root_senescence\4_egnet_pre_result(cv2)\split\no-taproot_Cmask_Splicing\2"
# save_file = r"D:\31890\Pictures\image\Root_senescence\4_egnet_pre_result(cv2)\split\no-taproot_Cmask_Splicing_Superpixel\2"
# if not os.path.exists(save_file):
#     os.makedirs(save_file)
# for name in tqdm.tqdm(os.listdir(image_file)):
#     image_path = os.path.join(image_file,name)
#     save_path = os.path.join(save_file,name.split('.')[0]+".png")
#     image_array = cv2.imread(image_path)
#     image_array = cv2.cvtColor(image_array,cv2.COLOR_BGR2RGB)
#     result= Superpixel_averaging(image_array,n_class=900)
#     cv2.imwrite(save_path,result)

## 主成分分析

#### 示例

In [ ]:
from time import time # 用于计算运行时间
import matplotlib.pyplot as plt 
import numpy as np
from matplotlib import offsetbox # 定义图形box的格式
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection) 

#加载手写数字数据集
digits = datasets.load_digits(n_class=6)
#手写图
X = digits.data
#类别
y = digits.target

# 首先定义函数画出二维空间中的样本点，输入参数：1.降维后的数据；2.图片标题
def plot_embedding(X, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min) # 对每一个维度进行0-1归一化，注意此时X只有两个维度
    
    plt.figure(figsize= (6,6)) # 设置整个图形大小
    ax = plt.subplot(111)
    colors = ['#5dbe80','#2d9ed8','#a290c4','#efab40','#eb4e4f','#929591']
    
    # 画出样本点
    for i in range(X.shape[0]): # 每一行代表一个样本
        plt.text(X[i, 0], X[i, 1], str(digits.target[i]),
                 #color=plt.cm.Set1(y[i] / 10.),
                 color=colors[y[i]],
                 fontdict={'weight': 'bold', 'size': 9})  # 在样本点所在位置画出样本点的数字标签
    
    # 在样本点上画出缩略图，并保证缩略图够稀疏不至于相互覆盖
    # 只有matplotlib 1.0版本以上，offsetbox才有'AnnotationBbox'，所以需要先判断是否有这个功能
    if hasattr(offsetbox, 'AnnotationBbox'): 
        shown_images = np.array([[1., 1.]])  # 假设最开始出现的缩略图在(1,1)位置上
        for i in range(digits.data.shape[0]):
            dist = np.sum((X[i] - shown_images) ** 2, 1) # 算出样本点与所有展示过的图片（shown_images）的距离
            if np.min(dist) < 4e-3: # 若最小的距离小于4e-3，即存在有两个样本点靠的很近的情况，则通过continue跳过展示该数字图片缩略图
                continue
            shown_images = np.r_[shown_images, [X[i]]] # 展示缩略图的样本点通过纵向拼接加入到shown_images矩阵中
            
            imagebox = offsetbox.AnnotationBbox(
                offsetbox.OffsetImage(digits.images[i], cmap=plt.cm.gray_r),
                X[i])
            ax.add_artist(imagebox)
            
    plt.xticks([]), plt.yticks([]) # 不显示横纵坐标刻度
    if title is not None: 
        plt.title(title) 
    plt.show()

t0 = time()
pca = decomposition.PCA(n_components=2)
X_pca = pca.fit_transform(X)
plot_embedding(X_pca,
               "Principal Components projection of the digits (time %.2fs)" %
               (time() - t0))
print (pca.explained_variance_ratio_) # 每一个成分对原数据的方差解释了百分之多少

#### 根系主成分分析

In [ ]:
import os
import tqdm
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import sklearn.cluster 
import pandas as pd
from sklearn import preprocessing
from time import time # 用于计算运行时间
import matplotlib.pyplot as plt 
import numpy as np
import cv2,os,tqdm
from matplotlib import offsetbox # 定义图形box的格式
from matplotlib.pyplot import MultipleLocator
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection) 

Image.MAX_IMAGE_PIXELS = None
image_file = r'D:\31890\Desktop\tranformer\senescence\shuailao_re'
def get_data(image_file):
    img_name_list = []
    i=0
    for name in tqdm.tqdm(os.listdir(image_file)):
        
        image_path = os.path.join(image_file,name)
        img = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
        img_flatten = img.flatten()
        img_name_list.append(name)
        img = np.expand_dims(img,axis=0)
        img_flatten = np.expand_dims(img_flatten,axis=0)
        if i==0:
            # img_group = img
            img_flatten_group = img_flatten
        else:
            # img_group = np.concatenate((img_group, img), axis = 0)
            img_flatten_group = np.concatenate((img_flatten_group, img_flatten), axis = 0) 
        i+=1
    # return img_group, img_flatten_group, img_name_list
    return img_flatten_group, img_name_list


img_flatten_group ,img_name_list =get_data(image_file)
# print(img_group.shape)
# print(img_flatten_group.shape)

t0 = time()
pca = decomposition.PCA(n_components=2)
X_pca = pca.fit_transform(img_flatten_group.astype(np.float32))

cluster_result = sklearn.cluster.AgglomerativeClustering(n_clusters=10).fit(X_pca)
img_class = cluster_result.labels_

In [ ]:
def plot_embedding(X, title=None):              ####—————— 首先定义函数画出二维空间中的样本点，输入参数：1.降维后的数据；2.图片标题
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)           ####—————— 对每一个维度进行0-1归一化，注意此时X只有两个维度
    
    plt.figure(figsize= (18,18))                ####—————— 设置整个图形大小
    ax = plt.subplot(111)
    colors = [  '#FF0000','#FF8C00','#FFE4B5','#9ACD32','#00FF00',      ####—————— 设置颜色矩阵
                '#00FFFF','#4169E1','#9400D3','#FF00FF','#708090']
    
    # 画出样本点
    plt.scatter(X[:, 0], X[:, 1])                                       ####—————— 画出样本点

    for i in range(X.shape[0]):                                         ####—————— 每一行代表一个样本
        plt.text(X[i, 0]+0.01, X[i, 1], img_name_list[i],
                 #color=plt.cm.Set1(y[i] / 10.),
                 color=colors[img_class[i]],
                #  horizontalalignment="left",
                 fontdict={'weight': 'bold', 'size': 12})               ####—————— 在样本点所在位置画出样本点的数字标签
    
    # 在样本点上画出缩略图，并保证缩略图够稀疏不至于相互覆盖
    # 只有matplotlib 1.0版本以上，offsetbox才有'AnnotationBbox'，所以需要先判断是否有这个功能
    # if hasattr(offsetbox, 'AnnotationBbox'): 
    #     shown_images = np.array([[1., 1.]])  # 假设最开始出现的缩略图在(1,1)位置上
    #     for i in range(img_flatten_group.shape[0]):
    #         dist = np.sum((X[i] - shown_images) ** 2, 1) # 算出样本点与所有展示过的图片（shown_images）的距离
    #         if np.min(dist) < 4e-3: # 若最小的距离小于4e-3，即存在有两个样本点靠的很近的情况，则通过continue跳过展示该数字图片缩略图
    #             continue
    #         shown_images = np.r_[shown_images, [X[i]]] # 展示缩略图的样本点通过纵向拼接加入到shown_images矩阵中
    #         plt_image = cv2.resize(img_group[i],(12,15))
    #         imagebox = offsetbox.AnnotationBbox(
    #             offsetbox.OffsetImage(plt_image, cmap=plt.cm.gray_r),
    #             X[i])
    #         ax.add_artist(imagebox)
    
    plt.xlim(-0.1,1.1)                              ####—————— 设置x轴的刻度限制
    plt.ylim(-0.1,1.1)                              ####—————— 设置y轴的刻度限制

    y_major_locator=MultipleLocator(0.1)            ####—————— 把y轴的刻度间隔设置为10，并存在变量里

    ax=plt.gca()                                    ####——————  ax为两条坐标轴的实例
    ax.yaxis.set_major_locator(y_major_locator)

    plt.xticks([]), plt.yticks([])                  ####—————— 不显示横纵坐标刻度
    if title is not None: 
        plt.title(title)                            ####—————— 设置图标的标题
    plt.savefig(r"D:\31890\Desktop\senescence\cluster.svg",dpi=200, bbox_inches='tight')    
    plt.show()
    
plot_embedding(X_pca,
               "Principal Components projection of the digits (time %.2fs)" %
               (time() - t0))
print (pca.explained_variance_ratio_)               ####——————  每一个成分对原数据的方差解释了百分之多少